In [ ]:
import warnings
import pandas as pd
import seaborn as sns
import numpy as np
import os
import json

from matplotlib import pyplot as plt
from dotenv import load_dotenv
from scipy import stats

from models.linear_regressions import Linear_reg
from scripts.frame_methods import scale_df, one_hot_dataframe

load_dotenv()
plt.style.use('Solarize_Light2')

# Setting default DPI, pulling it from dotenv if it exists, setting it on 100 if not

pc_dpi = int(os.getenv('DPI'))

if pc_dpi is None:
    pc_dpi = 100

warnings.filterwarnings("ignore")


<i>micro note : I use hashes freely for titles or important cells in mkdown, even though it sometimes doesnt change the layout, it eases access using Outline from Vscode</i>

# Feature engineering :

On cherche avant tout a maximiser l'efficacité du modèle en ajustant les données.On utilisera cela comme baseline `standard_scaler` de `sklearn` pour les benchmarks
<br>
Certaines features peuvent éventuellement être retouchées pour augmenter la précision de la modélisation. <br>

- On peut analyser la répartition et la déviation des variables linéaires continues. Selon ces paramètres, on appliquera un Standard_scaler, un passage au log, à la racine carrée ou en utilisant la méthode de Box Cox
- Les données concernant la taille des bâtiments (taille totale, taille bâtiments, taille parkings) peut être simplifiée : on peut utiliser la proportion de bâtiments et de parkings pour éviter les répétitions.
- Les features catégorielles non ordinales doivent être encodée (sauf utilisation de CatBoost, hors de cette étude), on utilisera One Hot Encoder.
- Conversion de "Year Built" vers "Building Age", cela devrait réduire le poids de cette variable (pour certains algorithmes) qui ne semblait pas avoir beaucoup d'influence sur les émissions de GàES ou la consommation énergétique durant l'analyse.
- Ajout d'une nouvelle métrique booléenne : Energy_Star_Certified (E* >= 75). Cela peut être une alternative à l'élimination totale de cette variable au profit d'un calcul plus simple. L'étude de classification de cette feature est hors du spectre de l'étude mais pourrait fournir une piste pour obtenir plus facilement les données E* sans les calculer.
<br>
<hr>
<br>


- ## 1 - Modification des variables : Building Age, Proportions (parking / building), Certification E*
- ## 2 - Analyse détaillée de la déviation et de la répartition des variables linéaires continues, application de la méthode optimale
    - _note : on sauvegarde un dataset standardisé pour comparaisons_ <br><br>
- ## 3 - Application de One Hot Encoder sur PrimaryPropertyType et Neighborhood
    - On s'attend à un très grand nombre de de nouvelles colonnes, on applique donc cette modification en dernier <br><br>
- ## 4 - Benchmark
    - On compare les métriques de la régression du nouveau dataset avec celles de base (std_scaled)
        - Si les métriques sont meilleure : On change de dataset
        - Sinon : on utilise le dataset sauvegardé dans 2 _note_ (avec variable standardisées et mises à l'échelle)

In [ ]:
raw_dataset = "./data/seattle_raw_data.csv"


In [ ]:
df_raw = pd.read_csv(raw_dataset)
df_raw.set_index("OSEBuildingID", inplace=True)


In [ ]:
df_raw.columns


In [ ]:
df_raw.head()


In [ ]:
df_raw["Neighborhood"] = df_raw["Neighborhood"].apply(str.upper)
# It will make sense when we apply OHE and save a lot of repetitions


### 1 : Modifications des variables :

- Âge
- Proportion de bâtiments/parking
- Certifié E* (booléen)


#### 1.1 : Age des batiments

In [ ]:
df_raw["BuildingAge"] = np.nan
for index, row in df_raw.iterrows():
    df_raw.loc[index, "BuildingAge"] = int(2022 - row["YearBuilt"])

df_raw.drop(columns=["YearBuilt"], inplace=True)


In [ ]:
# Check

df_raw["BuildingAge"].head()

# cf. cell 5 (previous call of .head() method), comparison checks up


#### 1.2 : Ajout d'une nouvelle variable : Energy Star Certified

In [ ]:
df_raw["EnergyStarCert"] = 0
for index, row in df_raw.iterrows():
    if row["ENERGYSTARScore"] >= 75:
        df_raw.loc[index, "EnergyStarCert"] = 1


In [ ]:
# Check :

df_raw[["EnergyStarCert", "ENERGYSTARScore"]].head(n=10)

# Coherent (0 if < 75, 1 if >= 75) , moving on


#### 1.3 : Modifications de variables : Proportion de parkings/batiments
- Il est inutile de fournir la surface dédiée a chaque activité au modèle : `surface = proportion_Building*taille_totale + proportion_Parking*taille_totale`
- L'utilisation d'un arrondi à 3 décimales causera dans certains cas `building_prop + parking_prop =/= 1`

In [ ]:
df_raw["Parking_proportion"] = np.nan
df_raw["Building_proportion"] = np.nan

for index, series in df_raw.iterrows():
    parking_prop = np.round(series["PropertyArea(SquareMetre)Parking"] / series["PropertyArea(SquareMetre)Total"], decimals=3)
    building_prop = np.round(series["PropertyArea(SquareMetre)Building(s)"] / series["PropertyArea(SquareMetre)Total"], decimals=3)
    df_raw.loc[index, "Parking_proportion"] = parking_prop
    df_raw.loc[index, "Building_proportion"] = building_prop

redundancies = ["LargestPropertyUseTypeArea(SquareMetre)", "PropertyArea(SquareMetre)Parking", "PropertyArea(SquareMetre)Building(s)"]

df_raw = df_raw.drop(columns=redundancies, errors="ignore")


In [ ]:
# Check : (if propP+propB =/= 1 : oops)

df_raw[["Parking_proportion", "Building_proportion"]].head()


### 2 - Analyse détaillée de la déviation et de la répartition des variables linéaires continues, application de la méthode optimale

#### 2.0 Standardisation et mise à l'échelle pour comparaison :

In [ ]:
## Scaling : 
## Doubling targets : Scaled if in X_matrix, but not as target

df_raw["target_GHGEmissionsIntensity(kgCO2e/ft2)"] = df_raw["GHGEmissionsIntensity(kgCO2e/ft2)"]
df_raw["target_SourceEUI(kWh/m2)"] = df_raw["SourceEUI(kWh/m2)"]


exempt_mod_cols = [
    "PrimaryPropertyType", "Neighborhood", "BuildingAge", "NumberofBuildings", "NumberofFloors",
    "ENERGYSTARScore", "Building_proportion", "Parking_proportion", "EnergyStarCert",
    "target_SourceEUI(kWh/m2)", "target_GHGEmissionsIntensity(kgCO2e/ft2)"  # Ignore targets
    ]

df_std_scaled = scale_df(df_raw, constant_col=exempt_mod_cols)


In [ ]:
# Check : 

df_std_scaled.head(n=2)


In [ ]:
col_subset_std = [
    "scaled_Electricity(kWh)", "scaled_GHGEmissionsIntensity(kgCO2e/ft2)", 
    "scaled_PropertyArea(SquareMetre)Total", "scaled_SourceEUI(kWh/m2)", 
    "scaled_NaturalGas(kWh)"
    ]

pairplot = sns.pairplot(
    data=df_std_scaled[col_subset_std]
)

pairplot.figure.set_dpi(100)

###
# Titles/Lables
for axes in pairplot.axes.flat:
    axes.set_ylabel("")
    axes.set_xlabel(axes.get_xlabel(), rotation=12)
pairplot.figure.suptitle("""Pairplot de distribution des variables\
 lineaires continues, standardisation et mise à l'échelle""")
#
###

plt.tight_layout()
plt.show()


##### Observations :

- Source_EUI, PropertyArea, semblent être relativement normalement réparties
- On enregistre la déviation des variables pour future comparaison


In [ ]:
skew_std_scaled = dict.fromkeys(col_subset_std)
for col in skew_std_scaled.keys():
    skew_std_scaled[col] = df_std_scaled[col].skew()


##### Copies des DataFrames
-> utilisation de pandas.DataFrame.copy() évite les problèmes d'interdépendance qui pourraient survenir avec df1=df2

#### 2.1 : Passage au log

In [ ]:
df_log = df_raw.copy(deep=True)
df_sqrt = df_raw.copy(deep=True)
df_bcx = df_raw.copy(deep=True)

col_subset = [
    "Electricity(kWh)", "GHGEmissionsIntensity(kgCO2e/ft2)", 
    "PropertyArea(SquareMetre)Total", "SourceEUI(kWh/m2)", "NaturalGas(kWh)"
    ]


In [ ]:
# check 1 : 

for col in col_subset:
    print((df_log[col] == 0).sum(), col)


In [ ]:
# just add 1 to all zeroes in log_data since log(1) = 0: 

for col in col_subset:
    df_log[col].replace(to_replace=0, value=1, inplace=True)


In [ ]:
# check 2 : 

for col in col_subset:
    print((df_log[col] == 0).sum(), col)



In [ ]:
log_data = df_log[col_subset]

for column in log_data.columns:
    log_data[column] = np.log(log_data[column])

pairplot = sns.pairplot(
    data=log_data
)

pairplot.figure.set_dpi(100)

###
# Titles/Lables
for axes in pairplot.axes.flat:
    axes.set_ylabel("")
    axes.set_xlabel(axes.get_xlabel(), rotation=12)
pairplot.figure.suptitle("Pairplot de distribution des variables lineaires continues, passage au log")
#
###

plt.tight_layout()
plt.show()


##### Observations : 
- Il est compliqué d'évaluer l'effet sur la déviation à l'œil nu : enregistrement pour comparaison
- Vérification nécessaire : df_raw ne contient pas de valeurs NaN, si des NaNs se manifestent, la cause est que la valeur est égale à 0
    - log(0) = -inf

In [ ]:
# checks : 

log_data.isna().sum()

# Seems we have a winner, might be a park or something that doesnt use energy / doesnt emmit GHG, will fill with 0


In [ ]:
skew_logged = dict.fromkeys(col_subset)
for col in skew_logged.keys():
    skew_logged[col] = log_data[col].skew(skipna=True)

skew_logged


##### Remarque :

- Le micro-fix consistant a remplacer les 0 par des 1 a fonctionné correctement
- Enregistrement de la déviation pour comparaison


#### 2.2 : Passage à la racine carrée

In [ ]:
sqrt_data = df_sqrt[col_subset]

for column in sqrt_data.columns:
    sqrt_data[column] = np.sqrt(sqrt_data[column])

pairplot = sns.pairplot(
    data=sqrt_data
)

pairplot.figure.set_dpi(100)

###
# Titles/Lables
for axes in pairplot.axes.flat:
    axes.set_ylabel("")
    axes.set_xlabel(axes.get_xlabel(), rotation=12)
pairplot.figure.suptitle("Pairplot de distribution des variables lineaires continues, passage a la racine carrée")
#
###

plt.tight_layout()
plt.show()


##### Observations : Il semble encore une fois compliqué de conclure sur la distribution à l'oeil nu

- On s'attend à voir deux résultats NaN : cf. hypothèse plus haut, on forcera le passage à 0
- On enregistre les valeurs de déviation

In [ ]:
skew_sqrt = dict.fromkeys(col_subset)
for col in skew_sqrt.keys():
    skew_sqrt[col] = sqrt_data[col].skew()

skew_sqrt


In [ ]:
# checks : 

sqrt_data.isna().sum()

# Called it ! Most likely the same one. Will perform same check before outputting to csv


#### 2.3 : Utilisation de la methode de Box Cox

In [ ]:
box_cox_data = df_bcx[col_subset]

for column in box_cox_data.columns:
    values = box_cox_data[column].values
    constant = abs(min(values)) + 0.000001
    new_values = values + constant
    new_values = stats.boxcox(new_values)
    box_cox_data[column] = new_values[0]

pairplot = sns.pairplot(
    data=box_cox_data
)

pairplot.figure.set_dpi(100)

###
# Titles/Lables
for axes in pairplot.axes.flat:
    axes.set_ylabel("")
    axes.set_xlabel(axes.get_xlabel(), rotation=12)
pairplot.figure.suptitle("Pairplot de distribution des variables lineaires continues, methode de Box Cox")
#
###

plt.tight_layout()
plt.show()


##### Observations : 
- Mêmes conclusions, on attend les chiffres pour procéder à une modification
- Re vérification des valeurs NaNs (on devrait n'avoir aucun NaN du fait de l'ajout d'une constante --> `constant = abs(min(values)) + 0.000001`

In [ ]:
skew_box_cox = dict.fromkeys(col_subset)
for col in skew_box_cox.keys():
    skew_box_cox[col] = box_cox_data[col].skew()


In [ ]:
# Check : 

box_cox_data.isna().sum()

# Called it again


In [ ]:
names = [name[7:] for name in skew_std_scaled.keys()]  # Removed "scaled_"
results = pd.DataFrame(data=skew_std_scaled.values(), columns=["standard_scaled"], index=names)
results["log"] = skew_logged.values()
results["sqrt"] = skew_sqrt.values()
results["box_cox"] = skew_box_cox.values()
results = results.T  # Transformation as index


In [ ]:
results


Utilisation de la valeur absolue minimale pour trouver la transformation adaptée à chaque variable :

In [ ]:
print("\n##############\n")

for col in results.columns:
    temp = abs(results[col])
    print(f"Minimal skew with {temp.idxmin()} for col {col[7:]} : skew = {results[col][temp.idxmin()]}")

print("\n##############\n")

for col in results.columns:
    temp = abs(results[col].drop(index=["box_cox"]))
    print(f" Minimal skew (without BCx) with {temp.idxmin()} for col {col[7:]} : skew = {results[col][temp.idxmin()]}")

print("\n##############\n")


### 3 : Application de One Hot Encoder sur PrimaryPropertyType et Neighborhood

#### 3.1 : Application des transformations sur les variables lineaires continues :
- 1 : Baseline en utilisant seulement standard_scaler
- 2 : Déviation la plus faible (incluant la méthode de Box Cox)
- 3 : Déviation la plus faible (exclusion de la méthode de Box Cox)
 

##### 3.1.1 : Standard scaler + OHE

In [ ]:
# df_std_scaled is already modified, lets just use OHE and we will be fine with that

ohe_subset = ["PrimaryPropertyType", "Neighborhood"]
ohe_prefix = ["Ptype", "Nbhood"]

df_std_scaled = one_hot_dataframe(
    dataframe=df_std_scaled,
    subset=ohe_subset,
    prefix=ohe_prefix,
    drop_og=True
    )


In [ ]:
# Checking : 

df_std_scaled.columns

# ||| should be plenty of OHE and no PrimaryPropertyType & Neighbourhood (im bad at words)
# |V| candidate 1 is ready, now for number 2 : 


##### 3.1.2 : Candidat 2 (utilisant la méthode de Box Cox)
- Minimal skew with box_cox for col Electricity(kWh) : skew = 0.14008000636298587
- Minimal skew with box_cox for col GHGEmissionsIntensity(kgCO2e/ft2) : skew = 0.039485740356718965
- Minimal skew with box_cox for col PropertyArea(SquareMetre)Total : skew = 0.1403831497723843
- Minimal skew with sqrt for col SourceEUI(kWh/m2) : skew = -0.018985565524061002
- Minimal skew with sqrt for col NaturalGas(kWh) : skew = 0.6566275254300923


In [ ]:
df_candidate_two = df_raw.copy(deep=True)

change_to_bcx = [
    "Electricity(kWh)", "GHGEmissionsIntensity(kgCO2e/ft2)", "PropertyArea(SquareMetre)Total"
    ]

change_to_sqrt = ["SourceEUI(kWh/m2)", "NaturalGas(kWh)"]


for column in change_to_bcx:
    values = df_candidate_two[column].values
    constant = abs(min(values)) + 0.000001
    new_values = values + constant
    new_values = stats.boxcox(new_values)
    df_candidate_two[column] = new_values[0]

for column in change_to_sqrt:
    df_candidate_two[column] = np.sqrt(df_candidate_two[column])



In [ ]:
# Applying OHE : 

df_candidate_two = one_hot_dataframe(
    dataframe=df_candidate_two,
    subset=ohe_subset,
    prefix=ohe_prefix,
    drop_og=True
)


In [ ]:
# Checking : 

df_candidate_two.columns

# ||| should be plenty of OHE and no PrimaryPropertyType & Neighbourhood (im bad at words)
# |V| candidate 2 is ready, now for number 3 :
 

##### 3.1.3 : Candidat 3 (en excluant la méthode de Box Cox)

- Minimal skew (without BCx) with sqrt for col Electricity(kWh) : skew = 0.8793089997087102
- Minimal skew (without BCx) with log for col GHGEmissionsIntensity(kgCO2e/ft2) : skew = -0.44455137612699647
- Minimal skew (without BCx) with log for col PropertyArea(SquareMetre)Total : skew = 0.5898718071984281
- Minimal skew (without BCx) with sqrt for col SourceEUI(kWh/m2) : skew = -0.018985565524061002
- Minimal skew (without BCx) with sqrt for col NaturalGas(kWh) : skew = 0.6566275254300923


In [ ]:
df_candidate_three = df_raw.copy(deep=True)

change_to_log = ["GHGEmissionsIntensity(kgCO2e/ft2)", "PropertyArea(SquareMetre)Total"]

change_to_sqrt = ["SourceEUI(kWh/m2)", "NaturalGas(kWh)", "Electricity(kWh)"]

for col in change_to_log:
    df_candidate_three[col].replace(to_replace=0, value=1, inplace=True)

for column in change_to_log:
    df_candidate_three[column] = np.log(df_candidate_three[column])

for column in change_to_sqrt:
    df_candidate_three[column] = np.sqrt(df_candidate_three[column])

df_candidate_three.fillna(value=0, inplace=True)


In [ ]:
# Applying OHE : 

df_candidate_three = one_hot_dataframe(
    dataframe=df_candidate_three,
    subset=ohe_subset,
    prefix=ohe_prefix,
    drop_og=True
)


In [ ]:
# Checking : 

df_candidate_two.columns

# ||| should be plenty of OHE and no PrimaryPropertyType & Neighbourhood (im bad at words)
# |V|
 

### 4 : Benchmark sur emission de GaES
- Kernel Fixé (via JSON)
- Memes methodes, ajustement eventuel des parametres
- Choix du meilleur dataset en fonction des meilleures metriques
- Evaluation sur Ridge uniquement (modele lineaire le plus performant selon nos données)

##### Chargement des indexes communs :

In [ ]:
# Loading known split, ids are unique building OSE id

with open("./data/splits_ghg.json", "r") as json_file:
    splits = json.load(json_file)

ids_train = splits["train"]
ids_test = splits["test"]


In [ ]:
## Setting targets & droplists : 

droplist_scaled = [
    "scaled_GHGEmissionsIntensity(kgCO2e/ft2)",  # Scaled target
    "target_SourceEUI(kWh/m2)",  # not to scale
    "EnergyStarCert"
    ]

droplist_generic = [
    "GHGEmissionsIntensity(kgCO2e/ft2)",  # Scaled target
    "target_SourceEUI(kWh/m2)",  # not to scale
    "EnergyStarCert"
    ]

target_ghg = "target_GHGEmissionsIntensity(kgCO2e/ft2)"

df_candidate_one = df_std_scaled.drop(columns=droplist_scaled)
df_candidate_two = df_candidate_two.drop(columns=droplist_generic)
df_candidate_three = df_candidate_three.drop(columns=droplist_generic)


##### Creation de trois objets de la classe Linear_reg :

In [ ]:
reg_candidate_one = Linear_reg(dataframe=df_candidate_one, target=target_ghg)
reg_candidate_two = Linear_reg(dataframe=df_candidate_two, target=target_ghg)
reg_candidate_three = Linear_reg(dataframe=df_candidate_three, target=target_ghg)


In [ ]:
# Overriding random splits :
# 1 : 

df_train_override_one = df_candidate_one[df_candidate_one.index.isin(ids_train)]
df_test_override_one = df_candidate_one[df_candidate_one.index.isin(ids_test)]

reg_candidate_one.force_split(
    df_train_ovr=df_train_override_one,
    df_test_ovr=df_test_override_one
)

###
# 2 :

df_train_override_two = df_candidate_two[df_candidate_two.index.isin(ids_train)]
df_test_override_two = df_candidate_two[df_candidate_two.index.isin(ids_test)]

reg_candidate_two.force_split(
    df_train_ovr=df_train_override_two,
    df_test_ovr=df_test_override_two
)

###
# 3 :

df_train_override_three = df_candidate_three[df_candidate_three.index.isin(ids_train)]
df_test_override_three = df_candidate_three[df_candidate_three.index.isin(ids_test)]

reg_candidate_three.force_split(
    df_train_ovr=df_train_override_three,
    df_test_ovr=df_test_override_three
)


In [ ]:
alphas_ridge = np.arange(0.1, 45, 0.05)
alphas_elnet = np.arange(42, 45, 1)  # We dont care about elnet, its the slowest and we wanna go fast
alphas_lasso = np.arange(1, 2, .5) # Same same

reg_candidate_one.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_ridge=alphas_ridge,
    alphas_lasso=alphas_lasso
)


In [ ]:
reg_candidate_one.ridge_plot()


In [ ]:
reg_one_perf = reg_candidate_one.ridge_table


In [ ]:
alphas_ridge = np.arange(1600, 2800, 0.1)  # different data different inputs via trial/error 
alphas_elnet = np.arange(42, 45, 1)  # We dont care about elnet, its the slowest and we wanna go fast
alphas_lasso = np.arange(1, 2, .5) # Same same

reg_candidate_two.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_ridge=alphas_ridge,
    alphas_lasso=alphas_lasso
)


In [ ]:
reg_candidate_two.ridge_plot()


In [ ]:
reg_two_perf = reg_candidate_two.ridge_table


In [ ]:
alphas_ridge = np.arange(5000, 6300, .1)  # different data different inputs via trial/error 
alphas_elnet = np.arange(42, 45, 1)  # We dont care about elnet, its the slowest and we wanna go fast
alphas_lasso = np.arange(1, 2, .5) # Same same

reg_candidate_three.execute_all(
    alphas_elnet=alphas_elnet,
    alphas_ridge=alphas_ridge,
    alphas_lasso=alphas_lasso
)


In [ ]:
reg_candidate_three.ridge_plot()


In [ ]:
reg_three_perf = reg_candidate_three.ridge_table


In [ ]:
# COMPARING : 
print("Ridge perf, standard_scaled data :")
reg_one_perf


In [ ]:
print("Ridge perf, lowest skew, BCX included :")
reg_two_perf


In [ ]:
print("Ridge perf, lowest skew, BCX excluded :")
reg_three_perf


# Conclusion & export : 

- A l'issue des benchmarks effectués sur les trois différentes méthodes de transformation de variables, l'utilisation de la standardisation/mise à l'échelle des variables linéaires continues se montre la plus précise sur la régression Ridge (score R2 plus haut et RMSE plus faible que les deux autres "candidats"). C'est cette modification qui sera exportée pour une étude plus approfondie.

<hr>

# Résumé des modifications : 

- Standardisation et mise à l'échelle des données linéaires continues
- Création de nouvelles variables : E* certified, BuildingAge
- Utilisation des proportions plutôt que des tailles (building+parking)

<hr>

## Export des données pour analyses approfondies :

In [ ]:
df_std_scaled.to_csv("./data/seattle_std_scaled.csv", sep=",")
